# Preprocessing Data

## Missing Value

In [1]:
# 1️⃣ Import library
import os
import pandas as pd

# 2️⃣ Pastikan folder hasil ada
os.makedirs("no2_results", exist_ok=True)

# 3️⃣ Baca data hasil unduhan (raw)
file_path = "no2_results/timeseries.csv"

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(f"✅ File ditemukan: {file_path}")
else:
    raise FileNotFoundError(f"❌ File tidak ditemukan di {file_path}")

# 4️⃣ Cek jumlah missing value sebelum interpolasi
missing_before = df["NO2"].isnull().sum()
print(f"\n📊 Jumlah data hilang (missing values) sebelum interpolasi: {missing_before}")

# 5️⃣ Pastikan kolom tanggal dalam format datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# 6️⃣ Lakukan interpolasi linear untuk kolom NO2
df["NO2"] = df["NO2"].interpolate(method="linear")

# 7️⃣ Hitung kembali jumlah missing setelah interpolasi
missing_after = df["NO2"].isnull().sum()
print(f"📈 Jumlah data hilang setelah interpolasi: {missing_after}")

# 8️⃣ Tampilkan ringkasan
print("\nRingkasan hasil interpolasi:")
print(f" - Total baris data : {len(df)}")
print(f" - Missing sebelum  : {missing_before}")
print(f" - Missing sesudah  : {missing_after}")
print(f" - Data berhasil dipulihkan: {missing_before - missing_after}")

# 9️⃣ Simpan hasil interpolasi ke file CSV baru
output_path = "no2_results/no2_interpolated_demak.csv"
df.to_csv(output_path, index=False)

print(f"\n✅ Data hasil interpolasi disimpan sebagai '{output_path}'")


✅ File ditemukan: no2_results/timeseries.csv

📊 Jumlah data hilang (missing values) sebelum interpolasi: 145
📈 Jumlah data hilang setelah interpolasi: 0

Ringkasan hasil interpolasi:
 - Total baris data : 395
 - Missing sebelum  : 145
 - Missing sesudah  : 0
 - Data berhasil dipulihkan: 145

✅ Data hasil interpolasi disimpan sebagai 'no2_results/no2_interpolated_demak.csv'


## Membentuk Data Supervised menggunakan 2,3,4,5 Hari sebelumnya

In [2]:
# 1️⃣ Import library
import pandas as pd
import os

# 2️⃣ Pastikan folder hasil ada
os.makedirs("no2_results", exist_ok=True)

# 3️⃣ Baca data interpolasi NO2
file_path = "no2_results/no2_interpolated_demak.csv"
new_df = pd.read_csv(file_path)

# 4️⃣ Pastikan urutan waktu benar
new_df = new_df.sort_values("date").reset_index(drop=True)

# 5️⃣ Hapus kolom tanggal (hanya pakai nilai NO2)
new_df = new_df.drop(columns=['date'])

# 6️⃣ Buat list untuk menyimpan hasil tiap lag
lag_list = [2, 3, 4, 5]
lag_dataframes = {}

# 7️⃣ Loop membuat data supervised untuk tiap lag
for n in lag_list:
    df_lag = new_df.copy()
    for i in range(1, n + 1):
        df_lag[f"t{i}"] = df_lag["NO2"].shift(i)
    df_lag = df_lag.dropna().reset_index(drop=True)
    
    # Simpan ke dictionary & file
    lag_dataframes[n] = df_lag
    output_path = f"no2_results/day{n}.csv"
    df_lag.to_csv(output_path, index=False)
    print(f"✅ Data supervised dengan t1–t{n} disimpan di: {output_path}")

# 8️⃣ Contoh tampilkan salah satu hasil (misal lag 5)
print("\n📊 Contoh hasil day5:")
print(lag_dataframes[5].head())


✅ Data supervised dengan t1–t2 disimpan di: no2_results/day2.csv
✅ Data supervised dengan t1–t3 disimpan di: no2_results/day3.csv
✅ Data supervised dengan t1–t4 disimpan di: no2_results/day4.csv
✅ Data supervised dengan t1–t5 disimpan di: no2_results/day5.csv

📊 Contoh hasil day5:
   feature_index       NO2        t1        t2        t3        t4        t5
0              0  0.000025  0.000026  0.000031  0.000027  0.000029  0.000025
1              0  0.000018  0.000025  0.000026  0.000031  0.000027  0.000029
2              0  0.000026  0.000018  0.000025  0.000026  0.000031  0.000027
3              0  0.000022  0.000026  0.000018  0.000025  0.000026  0.000031
4              0  0.000018  0.000022  0.000026  0.000018  0.000025  0.000026


### Memprediksi 3 hari kedepan

In [3]:
# 1️⃣ Import library
import pandas as pd
import os

# 2️⃣ Pastikan folder hasil ada
os.makedirs("no2_results", exist_ok=True)

# 3️⃣ Baca data interpolasi NO2
file_path = "no2_results/no2_interpolated_demak.csv"
new_df = pd.read_csv(file_path)

# 4️⃣ Pastikan urutan waktu benar
new_df = new_df.sort_values("date").reset_index(drop=True)

# 5️⃣ Hapus kolom tanggal (hanya pakai nilai NO2)
new_df = new_df.drop(columns=['date'])

# 6️⃣ Buat list untuk menyimpan hasil tiap lag
lag_list = [2, 3, 4, 5]
lag_dataframes = {}

# 7️⃣ Loop membuat data supervised multi-step untuk tiap lag
for n in lag_list:
    df_lag = new_df.copy()
    
    # Buat kolom lag (t1, t2, ..., tn)
    for i in range(1, n + 1):
        df_lag[f"t{i}"] = df_lag["NO2"].shift(i)
    
    # Tambahkan kolom target untuk 3 hari ke depan
    for j in range(1, 4):
        df_lag[f"NO2_t+{j}"] = df_lag["NO2"].shift(-j)
    
    # Hapus baris kosong akibat shift
    df_lag = df_lag.dropna().reset_index(drop=True)
    
    # Simpan ke dictionary & file
    lag_dataframes[n] = df_lag
    output_path = f"no2_results/day{n}_forecast3.csv"
    df_lag.to_csv(output_path, index=False)
    print(f"✅ Data supervised (lag {n}, forecast 3 hari) disimpan di: {output_path}")

# 8️⃣ Contoh tampilkan salah satu hasil (misal lag 5)
print("\n📊 Contoh hasil day5 (prediksi 3 hari ke depan):")
print(lag_dataframes[5].head())


✅ Data supervised (lag 2, forecast 3 hari) disimpan di: no2_results/day2_forecast3.csv
✅ Data supervised (lag 3, forecast 3 hari) disimpan di: no2_results/day3_forecast3.csv
✅ Data supervised (lag 4, forecast 3 hari) disimpan di: no2_results/day4_forecast3.csv
✅ Data supervised (lag 5, forecast 3 hari) disimpan di: no2_results/day5_forecast3.csv

📊 Contoh hasil day5 (prediksi 3 hari ke depan):
   feature_index       NO2        t1        t2        t3        t4        t5  \
0              0  0.000025  0.000026  0.000031  0.000027  0.000029  0.000025   
1              0  0.000018  0.000025  0.000026  0.000031  0.000027  0.000029   
2              0  0.000026  0.000018  0.000025  0.000026  0.000031  0.000027   
3              0  0.000022  0.000026  0.000018  0.000025  0.000026  0.000031   
4              0  0.000018  0.000022  0.000026  0.000018  0.000025  0.000026   

    NO2_t+1   NO2_t+2   NO2_t+3  
0  0.000018  0.000026  0.000022  
1  0.000026  0.000022  0.000018  
2  0.000022  0.00001